In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
# read conversion table and HS5 level sanction table
HS2022toHS2017 = pd.read_excel('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/HS2022toHS2017.xlsx', sheet_name='Conversions', dtype={'From HS2022': str, 'To HS2017': str}) 
HS5_sanct = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_4/HS5_sanct_short_ext_exp_v4.csv", dtype={'HS2017': str})
HS5_sanct[['Partial_HS5', 'Full_HS5','Transitory_HS5']] = HS5_sanct[['Partial_HS5', 'Full_HS5','Transitory_HS5']].fillna(value=0)
HS5_sanct.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
string_2017 = '201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712'
string_2018 = '201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812'
string_2019 = '201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912'
string_2020 = '202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012'
string_2021 = '202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112'
string_2022 = '202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212'
string_2023 = '202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312'
string_2024 = '202401,202402,202403,202404,202405'

In [4]:
string_dates = [string_2017, string_2018, string_2019, string_2020, string_2021, string_2022, string_2023, string_2024]

In [5]:
import requests
import comtradeapicall
import time
import datetime
subscription_key = '94ee5163870949e1aa6fe904f64e241d' # comtrade api subscription key (from comtradedeveloper.un.org)
directory = '/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Comtrade_v1'  # output directory for downloaded files 
#proxy_url = '<PROXY URL>'  # optional if you need proxy url
# Call preview final data API to a data frame, max to 500 records, no subscription key required
# This example: Australia imports of commodity code 91 in classic mode in May 2022
#typeCode: goods or services
#freqCode: annual or month, eg '202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112'
#period: eg '200001,200002,200003'
#reporterCode: 97 - EU
#partnerCode: 
# 398 - Kzt
# 51 - Armenia
# 417 - Kyrgyzstan
# 792 - Türkiye
# 156 - China
# 756 or 757 - Switzerland
# 643 - Russia

In [6]:
def get_data_exp(params):
    data = comtradeapicall.getFinalData(subscription_key, typeCode='C', freqCode='M', clCode='HS', period=params[0],
                                        reporterCode=params[1], cmdCode=None, flowCode='X', partnerCode=params[2],
                                        partner2Code=None,
                                        customsCode=None, motCode=None, maxRecords=250000, format_output='JSON',
                                        aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True)
    return data

#### Download EEU3 export to Russia

In [ ]:
ECU3 = get_data_exp([string_dates[0], '398,51,417', '643'])


In [11]:
for years in tqdm(string_dates[1:]):
    ECU3_new = get_data_exp([years, '398,51,417', '643'])
    ECU3 = ECU3.append(ECU3_new)


100%|██████████| 7/7 [04:43<00:00, 40.54s/it]


In [14]:
ECU3_W = get_data_exp([string_dates[0], '398,51,417', '0'])

In [16]:
for years in tqdm(string_dates[1:]):
    ECU3_W_new = get_data_exp([years, '398,51,417', '0'])
    ECU3_W = ECU3_W.append(ECU3_W_new)


100%|██████████| 7/7 [08:59<00:00, 77.01s/it]


In [17]:
ECU3 = ECU3.append(ECU3_W)

In [19]:
# fine
#pd.crosstab(index=ECU3['classificationCode'], columns=ECU3['refYear'])

refYear,2017,2018,2019,2020,2021,2022,2023,2024
classificationCode,,,,,,,,
H5,95781,109906,94933,97670,101411,0,0,0
H6,0,0,0,0,0,127304,157290,44014


In [89]:
(95781 + 109906 + 94933 + 97670 + 101411 + 127304) / 3

209001.66666666666

In [20]:
# keep only needed col-s
#comtrade_cols = ['period', 'partnerCode', 'partnerISO', 'classificationCode', 'cmdCode', 'primaryValue']
comtrade_cols = ['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']
ECU3_short = ECU3[comtrade_cols]
ECU3_short = ECU3_short.loc[ECU3_short['cmdCode'].apply(len) == 6]
ECU3_short.reset_index(inplace=True)
ECU3_short.drop_duplicates(keep='first', inplace=True) # drop duplicates if were downloaded accidentally

In [22]:
# ECU3_short_agg = ECU3_short.groupby(['period', 'cmdCode', 'partnerCode'])['primaryValue'].sum().reset_index() # sums over countries
# ECU3_short_agg['reporterCode'] = 'ECU3'
# ECU3_short_agg = ECU3_short_agg[['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]

In [26]:
regres_countr = [643] # Rus
regres_countr_data = ECU3_short_agg.loc[ECU3_short_agg['partnerCode'].apply(lambda x: x in regres_countr)]
regres_countr_data_agg = regres_countr_data.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index()


In [28]:
RestWorld = ECU3_short_agg.loc[ECU3_short_agg['partnerCode'] == 0]
RestWorld = pd.merge(RestWorld, regres_countr_data_agg, on=['period', 'cmdCode'], how='left')
RestWorld['primaryValue_y'].fillna(value=0, inplace=True)
RestWorld['primaryValue'] = RestWorld['primaryValue_x'] - RestWorld['primaryValue_y']
RestWorld['partnerCode'] = "RW_ECU3_1"
RestWorld['reporterCode'] = 'ECU3'
RestWorld = RestWorld[['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]

ECU3_short_agg = ECU3_short_agg.append(RestWorld)

In [30]:
# check on one example that RW is correct
#ECU3_short_agg.loc[(ECU3_short_agg['period'] == '202405') & (ECU3_short_agg['cmdCode'] == '961900')]

,period,reporterCode,partnerCode,cmdCode,primaryValue
362814,202405,ECU3,0,961900,128087.03
362815,202405,ECU3,643,961900,117543.65
212405,202405,ECU3,RW_ECU3_1,961900,10543.38


In [84]:
#ECU3_short_agg.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/ECU3_short_agg_exp.csv")

#### Download China export to Russia, EEU3

In [33]:
China = get_data_exp([string_dates[0], '156', '643,0'])


In [37]:
for years in tqdm(string_dates[1:]):
    China_new = get_data_exp([years, '156', '643,0'])
    China = China.append(China_new)


100%|██████████| 7/7 [11:58<00:00, 102.61s/it]


In [40]:
China_ECU3 = get_data_exp([string_dates[0], '156', '398,51,417'])

In [41]:
for years in tqdm(string_dates[1:]):
    China_ECU3_new = get_data_exp([years, '156', '398,51,417'])
    China_ECU3 = China_ECU3.append(China_ECU3_new)


100%|██████████| 7/7 [05:58<00:00, 51.25s/it]


In [44]:
China = China.append(China_ECU3)

In [45]:
# fine
pd.crosstab(index=China['classificationCode'], columns=China['refYear'])

refYear,2017,2018,2019,2020,2021,2022,2023
classificationCode,,,,,,,
H5,175989,179720,184288,173684,181928,0,0
H6,0,0,0,0,0,196143,190063


In [46]:
# keep only needed col-s
#comtrade_cols = ['period', 'partnerCode', 'partnerISO', 'classificationCode', 'cmdCode', 'primaryValue']
comtrade_cols = ['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']
China_short = China[comtrade_cols]
China_short = China_short.loc[China_short['cmdCode'].apply(len) == 6]
China_short.reset_index(inplace=True)
China_short.drop_duplicates(keep='first', inplace=True) # drop duplicates if were downloaded accidentally

In [11]:
#China_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/China_short_exp.csv")
China_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/China_short_exp.csv", dtype={'cmdCode': str})
China_short.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
# check all cmdCode have 6 digits
#China_short['cmdCode'].apply(len).unique()

In [12]:
#China_ECU3_short = China_short.loc[China_short['partnerCode'].apply(lambda x: x in [398,51,417])]

In [13]:
# China_ECU3_short_agg = China_ECU3_short.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index() # sums over countries
# China_ECU3_short_agg['partnerCode'] = 'ECU3'
# China_ECU3_short_agg['reporterCode'] = 156
# China_ECU3_short_agg = China_ECU3_short_agg[['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]

In [14]:
# China_short = China_short.loc[China_short['partnerCode'].apply(lambda x: x not in [398,51,417])]
# China_short = China_short.append(China_ECU3_short_agg)

In [15]:
regres_countr = [643, 398, 51, 417] # Rus, ECU3
regres_countr_data = China_short.loc[China_short['partnerCode'].apply(lambda x: x in regres_countr)]
regres_countr_data_agg = regres_countr_data.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index()


In [16]:
RestWorld = China_short.loc[China_short['partnerCode'] == 0]
RestWorld = pd.merge(RestWorld, regres_countr_data_agg, on=['period', 'cmdCode'], how='left')
RestWorld['primaryValue_y'].fillna(value=0, inplace=True)
RestWorld['primaryValue'] = RestWorld['primaryValue_x'] - RestWorld['primaryValue_y']
RestWorld['partnerCode'] = "RW_China_2"
RestWorld['reporterCode'] = 156
RestWorld = RestWorld[['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]

China_short = China_short.append(RestWorld)

In [44]:
#China_short['partnerCode'].unique()
#China_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/China_short_exp_v2.csv")


In [13]:
#1000/1010, 10/1010, 1010/1021, 11/1021
#12 * 6 * 2500 * 6

#### Download Turkiye export to Russia, EEU3

In [47]:
Turkiye = get_data_exp([string_dates[0], '792', '643,0'])


In [49]:
for years in tqdm(string_dates[1:]):
    Turkiye_new = get_data_exp([years, '792', '643,0'])
    Turkiye = Turkiye.append(Turkiye_new)


100%|██████████| 7/7 [45:57<00:00, 393.88s/it]


In [54]:
Turkiye_ECU3 = get_data_exp([string_dates[0], '792', '398,51,417'])

In [55]:
for years in tqdm(string_dates[1:]):
    Turkiye_ECU3_new = get_data_exp([years, '792', '398,51,417'])
    Turkiye_ECU3 = Turkiye_ECU3.append(Turkiye_ECU3_new)


100%|██████████| 7/7 [04:11<00:00, 35.88s/it]


In [59]:
Turkiye = Turkiye.append(Turkiye_ECU3)

In [60]:
# fine 
pd.crosstab(index=Turkiye['classificationCode'], columns=Turkiye['refYear'])

refYear,2017,2018,2019,2020,2021,2022,2023,2024
classificationCode,,,,,,,,
H5,121856,125589,128519,132315,144254,0,0,0
H6,0,0,0,0,0,153569,164199,67181


array([ 51, 398, 417])

In [68]:
# keep only needed col-s
#comtrade_cols = ['period', 'partnerCode', 'partnerISO', 'classificationCode', 'cmdCode', 'primaryValue']
comtrade_cols = ['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']
Turkiye_short = Turkiye[comtrade_cols]
Turkiye_short = Turkiye_short.loc[Turkiye_short['cmdCode'].apply(len) == 6]
Turkiye_short.reset_index(inplace=True)
Turkiye_short.drop_duplicates(keep='first', inplace=True) # drop duplicates if were downloaded accidentally

In [5]:
#Turkiye_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/Turkiye_short_exp.csv")
Turkiye_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/Turkiye_short_exp.csv", dtype={'cmdCode': str})
Turkiye_short.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
# check all cmdCode have 6 digits
#Turkiye_short['cmdCode'].apply(len).unique()

In [6]:
#Turkiye_ECU3_short = Turkiye_short.loc[Turkiye_short['partnerCode'].apply(lambda x: x in [398,51,417])]

In [7]:
# Turkiye_ECU3_short_agg = Turkiye_ECU3_short.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index() # sums over countries
# Turkiye_ECU3_short_agg['partnerCode'] = 'ECU3'
# Turkiye_ECU3_short_agg['reporterCode'] = 792
# Turkiye_ECU3_short_agg = Turkiye_ECU3_short_agg[['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]


In [8]:
# Turkiye_short = Turkiye_short.loc[Turkiye_short['partnerCode'].apply(lambda x: x not in [398,51,417])]
# Turkiye_short = Turkiye_short.append(Turkiye_ECU3_short_agg)

In [9]:
regres_countr = [643, 398,51,417] # Rus, ECU3
regres_countr_data = Turkiye_short.loc[Turkiye_short['partnerCode'].apply(lambda x: x in regres_countr)]
regres_countr_data_agg = regres_countr_data.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index()


In [10]:
RestWorld = Turkiye_short.loc[Turkiye_short['partnerCode'] == 0]
RestWorld = pd.merge(RestWorld, regres_countr_data_agg, on=['period', 'cmdCode'], how='left')
RestWorld['primaryValue_y'].fillna(value=0, inplace=True)
RestWorld['primaryValue'] = RestWorld['primaryValue_x'] - RestWorld['primaryValue_y']
RestWorld['partnerCode'] = "RW_Turkiye_2"
RestWorld['reporterCode'] = 792
RestWorld = RestWorld[['period', 'reporterCode', 'partnerCode', 'cmdCode', 'primaryValue']]

Turkiye_short = Turkiye_short.append(RestWorld)
#Turkiye_short.drop(['index'], axis=1, inplace=True)


### Combine all data

In [17]:
#Turkiye_short['partnerCode'].unique()
ECU3_short_agg = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v2/ECU3_short_agg_exp.csv", dtype={'cmdCode': str})
ECU3_short_agg.drop(['Unnamed: 0'], axis=1, inplace=True)


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
database = ECU3_short_agg.append([China_short, Turkiye_short])
database['partnerCode'] = database['partnerCode'].apply(str)
database = database.loc[database['partnerCode'] != '0']
# database.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
database.drop_duplicates(keep='first', inplace=True)
database['period'] = database['period'].apply(str)

In [65]:
#database_2022_2023_HS5['reporterCode'].unique()

In [19]:
# append datasets according to HS5 and HS6 classif
database_2017_2021 = database.loc[database['period'] < '202201']
database_2022_2023 = database.loc[database['period'] >= '202201']

In [20]:
# convet HS6 to HS5 and group HS6 flows within same month-HS5 group
database_2022_2023_HS5 = pd.merge(database_2022_2023, HS2022toHS2017, left_on='cmdCode', right_on='From HS2022', how='left')
database_2022_2023_HS5 = database_2022_2023_HS5[['period', 'reporterCode', 'partnerCode', 'primaryValue', 'To HS2017']].groupby(['period', 'reporterCode', 'partnerCode', 'To HS2017'])['primaryValue'].sum().reset_index()


In [21]:
# rename to have same column names
database_2017_2021.rename(columns={'cmdCode':'cmdCode_HS5'}, inplace=True)
database_2022_2023_HS5.rename(columns={'To HS2017':'cmdCode_HS5'}, inplace=True)

database_HS5 = database_2017_2021.append(database_2022_2023_HS5)


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
database_HS5_sanct = pd.merge(database_HS5, HS5_sanct, left_on='cmdCode_HS5', right_on='HS2017', how='left')
database_HS5_sanct.drop(['HS2017', 'HS2022'], axis=1, inplace=True)
database_HS5_sanct['period'] = database_HS5_sanct['period'].apply(lambda x: pd.to_datetime(x, format='%Y%m')) # convert to timestamp
database_HS5_sanct['Date_HS5'] = database_HS5_sanct['Date_HS5'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d') if x==x else x)
database_HS5_sanct['Date_HS5_similar'] = database_HS5_sanct['Date_HS5_similar'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d') if x==x else x)


In [23]:
print(database_HS5_sanct['Sanct_HS5'].isna().sum())
database_HS5_sanct = database_HS5_sanct.loc[database_HS5_sanct['Sanct_HS5'].notna()]
database_HS5_sanct['partnerCode'] = database_HS5_sanct['partnerCode'].apply(str)

6422


#### Prepare for regressions

In [81]:
# regres_countr + ['RW2']
database_HS5_sanct['partnerCode'].unique()


array(['643', 'RW_ECU3_1', '51', '398', '417', 'RW_China_2',
       'RW_Turkiye_2'], dtype=object)

In [24]:
#database_reg = database_HS5_sanct.loc[(database_HS5_sanct['partnerCode'] != 0) & (database_HS5_sanct['partnerCode'] != 156)]
regres_countr = ['643', 'RW_ECU3_1', '51', '398', '417', 'RW_China_2', 'RW_Turkiye_2']
database_reg = database_HS5_sanct.loc[database_HS5_sanct['partnerCode'].apply(lambda x: x in regres_countr + ['RW2'])]


In [25]:
def periodtoint(period):
    if period == period: return period.year*12 + period.month - 24205 # 24240 is number of months before 2020-11-01
    else: return 0

database_reg.sort_values(by=['period'], inplace=True)
#ECU3_sanct_1['Date_HS5'] = ECU3_sanct_1['Date_HS5'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')) # convert to timestamp
database_reg['Date_HS5_int'] = database_reg['Date_HS5'].apply(periodtoint)
database_reg['period_int'] = database_reg['period'].apply(periodtoint)
database_reg['log_primaryValue'] = np.log(database_reg['primaryValue'])


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [26]:
database_reg['Sanct_t'] = (database_reg['period'] > database_reg['Date_HS5']).apply(int)
database_reg['Similar_t'] = (database_reg['period'] > database_reg['Date_HS5_similar']).apply(int)
database_reg['Sanct_t_int'] = (database_reg['period_int'] > database_reg['Date_HS5_int']).apply(int) # becareful, this might be falacious
database_reg['partnerCode'] = database_reg['partnerCode'].apply(str)
database_reg['reporterCode'] = database_reg['reporterCode'].apply(str)

# status at t: Partial_t = 1 if partially sanctioned at t, o o/w; Full_t = 1 if fully sanctioned at t, o o/w;
# Thus transitory code has Partial_t = 1 when partially sanctioned and Full_t = 1 when fully
database_reg['Partial_t'] = database_reg['Partial_HS5'] * database_reg['Sanct_t'] + database_reg['Transitory_HS5'] * (((database_reg['period'] > database_reg['Transitory_firstpatial_HS5']) & (database_reg['period'] <= database_reg['Transitory_firstfull_HS5'])).apply(int))
database_reg['Full_t'] = database_reg['Full_HS5'] * database_reg['Sanct_t'] + database_reg['Transitory_HS5'] * ((database_reg['period'] > database_reg['Transitory_firstfull_HS5']).apply(int))


In [27]:
# becareful as 0 time_to_treat will code both, treatment period and non-sanct products => always use with sanctioned indicator to distinguish
database_reg['time_to_treat'] = database_reg['period_int'] - database_reg['Date_HS5_int']
database_reg.loc[database_reg['Sanct_HS5'] == 0, 'time_to_treat'] = 0

In [28]:
def time_to_treat(x):
    if x < -12: return -13
    elif x > 12: return 13
    else: return x
    
database_reg['time_to_treat_ES1'] = database_reg['time_to_treat'].apply(time_to_treat)    

In [29]:
# split on exporter datasets
database_reg_ECU3_agg = database_reg.loc[database_reg['reporterCode'] == 'ECU3']
database_reg_China = database_reg.loc[database_reg['reporterCode'] == '156']
database_reg_Turkiye = database_reg.loc[database_reg['reporterCode'] == '792']

database_reg_ECU3_agg[["Rus", 'RW1']] = pd.get_dummies(database_reg_ECU3_agg['partnerCode'])
database_reg_China[['Kaz', 'Kyr', 'Arm', "Rus", 'RW2']] = pd.get_dummies(database_reg_China['partnerCode'])
database_reg_Turkiye[['Kaz', 'Kyr', 'Arm', "Rus", 'RW2']] = pd.get_dummies(database_reg_Turkiye['partnerCode'])


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [30]:
# dummies for ECU3 and FSU3
database_reg_China['ECU3'] = database_reg_China['Kaz'] + database_reg_China['Kyr'] + database_reg_China['Arm']
database_reg_China['ECU3'] = database_reg_China['ECU3'].apply(lambda x: int(x>0))

database_reg_Turkiye['ECU3'] = database_reg_Turkiye['Kaz'] + database_reg_Turkiye['Kyr'] + database_reg_Turkiye['Arm']
database_reg_Turkiye['ECU3'] = database_reg_Turkiye['ECU3'].apply(lambda x: int(x>0))


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [119]:
pd.get_dummies(database_reg_China['partnerCode'])

,398,417,51,643,RW_China_2
223052,0,0,0,1,0
223053,0,0,0,1,0
223054,0,0,0,1,0
223055,0,0,0,1,0
223056,0,0,0,1,0
...,...,...,...,...,...
1976069,0,0,0,0,1
1976068,0,0,0,0,1
1976067,0,0,0,0,1
1976066,0,0,0,0,1


In [36]:
print(database_reg_China['ECU3'].sum())
database_reg_China
(database_reg_China['partnerCode'] == '51').sum()

297214


46522

In [37]:
print(database_reg_Turkiye['ECU3'].sum())
#(database_reg_Turkiye['partnerCode'] == '51').sum()

200689


In [117]:
database_reg_ECU3_agg.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v3/database_reg_ECU3_agg.csv")
database_reg_China.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v3/database_reg_China.csv")
database_reg_Turkiye.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_9v3/database_reg_Turkiye.csv")


In [113]:

#China_short.loc[China_short['partnerCode'] == 51]

#pd.crosstab(index=China_short.loc[China_short['partnerCode'] == 51]['reporterCode'], columns=China_short.loc[China_short['partnerCode'] == 51]['period'])

#China_short.loc[China_short['partnerCode'] == 51]

#pd.crosstab(index=database_reg_China['partnerCode'], columns=database_reg_China['period'])

In [116]:
database_reg_China['time_to_treat_ES1'].unique()


array([  0, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,
        -1,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13])

In [89]:
#database_reg.loc[database_reg['reporterCode'].isna()]



In [ ]:
database_reg.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/database_exp_reg_v5_RW2.csv")

